# Zynq-7000 Porting Tutorial

This document describes how to go about porting the PYNQ software to other boards based on Zynq-7000. It uses the Zybo board as an example but the same steps can be applied to any similar board.

## Overview of the process

The main task is assembling the required boot files and configurations. In particular we need the following
 * The processing system configuration
 * The U-boot version and configuration
 * The Linux kernel and configuration
 * A bitstream to load at boot time

Once we have the boot files we can reuse most the root filesystem configuration from the PYNQ-Z1 to create the full configuration of the image.

## Boot files

### config

Most of the process of generating the boot files is automated however we need to provide the configurations.

First we'll create a new directory in the `boot_configs` directory.

Looking through the config file of the PYNQ-Z1-defconfig boot configuration gives us the list of files that we need to provide to complete the port so we'll copy it in as a template and go about modifying it

```
BOOT_CONFIG_DIR := $(dir $(word $(words $(MAKEFILE_LIST)),$(MAKEFILE_LIST)))
BOOT_CONFIG_ABS = $(CURDIR)/${BOOT_CONFIG_DIR}

LINUX_REPO ?= https://github.com/Xilinx/linux-xlnx.git
LINUX_COMMIT ?= 4e93098735caeb862042d6bf601503666cdc0b69

UBOOT_REPO ?= https://github.com/Digilent/u-boot-digilent.git
UBOOT_COMMIT ?= e2382ceaffc0c36dc21c01576c1e454af0e01ca1

BOARD = Pynq-Z1
BOARD_PART = xc7z020clg400-1
PS_CONFIG_TCL ?= ${BOOT_CONFIG_ABS}/ps7_config.tcl
BOARD_CONSTRAINTS ?= ${BOOT_CONFIG_ABS}/constraints.xdc

LINUX_CONFIG ?= ${BOOT_CONFIG_ABS}/kernel.config
UBOOT_CONFIG ?= ${BOOT_CONFIG_ABS}/u-boot.config
BOARD_DTSI ?= ${BOOT_CONFIG_ABS}/pynq.dtsi

include ${BOOT_CONFIG_DIR}/../common/Zynq7000.makefile

```

The first two lines are boiler-plate code to allow us to access the current directory within the Makefile.

The next two lines are the Linux kernel repository and commit to use. This corresponds to the 2016.4 PetaLinux release and we can keep this the same for our port.

The next two lines are the u-boot repository to use. The Zybo is supported by the Xilinx repository so we can change this to point at the 2016.4 release of the u-boot from Xilinx, replacing the two lines with:

```
UBOOT_REPO ?= https://github.com/Xilinx/u-boot-xlnx.git
UBOOT_COMMIT ?= 0b94ce5ed4a6c2cd0fec7b8337e776b03e387347
```

Next we have the `BOARD` variable which  will be used when installing the PYNQ software and appear in the image as an environment variable. This gets changed to `Zybo`. The `BOARD_PART` variable also needs to be updated to `xc7z010clg400-1`

There is one other variable that we should add which says which boot bitstream should be used (`BOOT_BITSTREAM`). By default the Makefile will look for the base bitstream of the board and as we don't have one we should give it something else. In this case we'll tell it to use `empty.bit` - a bitstream we'll create later.

The rest of the file is just referencing other files in the directory so we will take each one in turn and the last line imports the main Zynq-7000 Makefile which builds the files.

The end result is a file that looks like:

```
BOOT_CONFIG_DIR := $(dir $(word $(words $(MAKEFILE_LIST)),$(MAKEFILE_LIST)))
BOOT_CONFIG_ABS = $(CURDIR)/${BOOT_CONFIG_DIR}

LINUX_REPO ?= https://github.com/Xilinx/linux-xlnx.git
LINUX_COMMIT ?= 4e93098735caeb862042d6bf601503666cdc0b69

UBOOT_REPO ?= https://github.com/Xilinx/u-boot-xlnx.git
UBOOT_COMMIT ?= 0b94ce5ed4a6c2cd0fec7b8337e776b03e387347

BOARD = Zybo
BOARD_PART = xc7z010clg400-1
PS_CONFIG_TCL ?= ${BOOT_CONFIG_ABS}/ps7_config.tcl
BOARD_CONSTRAINTS ?= ${BOOT_CONFIG_ABS}/constraints.xdc

LINUX_CONFIG ?= ${BOOT_CONFIG_ABS}/kernel.config
UBOOT_CONFIG ?= ${BOOT_CONFIG_ABS}/u-boot.config
BOARD_DTSI ?= ${BOOT_CONFIG_ABS}/pynq.dtsi
BOOT_BITSTREAM := ${BOOT_CONFIG_ABS}/empty.bit

include ${BOOT_CONFIG_DIR}/../common/Zynq7000.makefile
```

### ps7_config.tcl

This file contains the configuration of the processing system and is used for generating the First stage bootloader and device tree. To create it, create an empty project for the board, add the processing system and run block automation if necessary. You should have a design with just the processing system. All AXI interfaces on the PS should be disabled. Double click on the processing system, click presets on the toolbar and then Save Configuration. The configuration can be named anything and the resulting tcl file becomes ps7_config.tcl. While the project is open, we can build a bitstream to create empty.bit

### constraints.xdc

This isn't needed for the Zybo and can be empty.

### kernel.config

This file is the Linux kernel configuration to use. In this case we can re-use the same config as the PYNQ-Z1 board but in general the configuration can be obtained by following the instructions provided with the board. Normally this involves cloning a Linux repository and running `make ARCH=arm xilinx_zynq_defconfig`. This will create a plain configuration with all of the devices needed for the chip. The PYNQ framework requires that a couple of additional drivers also be added: the xlnk and uio platform drivers. These can be enabled by being in the Linux source directory and running `make ARCH=arm menuconfig` and enabling Device Drivers->Staging drivers->Xilinx APF Accelerator driver and Device Drivers->Userspace I/O drivers->Userspace I/O platform driver with generic IRQ handling.

Once you have a kernel configuration copy .config in the source tree to kernel.config in the boot configuration folder.

### u-boot.config

This is made in a similar way by cloning the u-boot repository and running `make zynq_zybo_defconfig`. The resulting .config can be copied to u-boot.config. If additional configuration is required it can be done inside the u-boot repository.

### pynq.dtsi

This file adds the additional drivers needed for the PYNQ software framework. It can be copied from the PYNQ-Z1 configuration.

## Root file system

The only thing we need to change from the PYNQ-Z1 is the boot_leds script which will not work on Zybo. To create a new rootfs configuration we can copy the rootfs_configs/Pynq-Z1-Wily to rootfs_configs/Zybo-Wily and remove boot_leds from the STAGE3_PACKAGES variable. More details of root filesystem configuration can be found in the packages directory.

## Release configuration

The final thing we need is a release which is in the releases folder. This is a combination of a boot configuration and rootfs configuration. For the Zybo we will have the Zybo-Wily.config with the two lines

```
BOOT_CONFIG=Zybo-defconfig
ROOTFS_CONFIG=Zybo-Wily
```

## Building

The final stage is to build the whole thing by running `make RELEASE=Zybo-Wily`. This will take several hours and result in a rootfs.img file ready for loading on to an SD card. The boot files can be made separately by running `make RELEASE=Zybo-Wily boot_files` with the files ending up in .build/boot.